<a href="https://colab.research.google.com/github/tannerunaveen4/hds5210-2023/blob/main/midterm/midterm_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [99]:
def priest(sex, age, respiratory_rate_per_minute, oxygen_saturation,
           heart_rate, systolic_BP,  temperature, alertness, inspired_oxygen, performance_status):
  """(str, int, float, float, int, float, float, str, str, str)->float
  Calculate PRIEST Clinical Severity Score.
  example:
  >>> priest('male', 50, 24, 0.80, 80, 120, 37, 'alert', 'room air', 'good')
  >>> priest('male', 50, 24, 0.80, 80, 120, 37, 'alert', 'air', 'unrestricted normal activity')
  0.22
  >>> priest('female', 66, 26, 0.90, 140, 80, 34, 'Confused or not alert', 'Supplemental oxygen', 'Bed/chair bound, no self-care')
  0.99
  >>> priest('female', 66, 26, 0.90, 140, 80, 34, 'Confused or not alert', 'Supplemental oxygen', 'Unrestricted normal activity')
  0.59
  >>> priest('female', 66, 26, 0.90, 140, 80, 34, 'Confused or not alert', 'Supplemental oxygenss', 'Unrestricted normal activity')
  """
  #Trensforming all the string input values lowercase
  sex = sex.lower()
  alert = alertness.lower()
  inspired_oxygen = inspired_oxygen.lower()
  performance_status = performance_status.lower()
  points = 0

  #checking if the input values are correct for sex case
  if sex not in ['male', 'female']:
    return None
  elif sex == 'female':
    points += 0
  elif sex == 'male':
    points += 1

  #checking for age case in it's categories
  if age >= 16 and age < 50 :
    points += 0
  elif age >= 50 and age < 66:
    points += 2
  elif age >= 66 and age <= 80 :
    points += 3
  elif age > 80:
    points += 4


  #checking for respiratory respiratory rate in breaths per minute
  if respiratory_rate_per_minute < 9:
    points += 3
  elif respiratory_rate_per_minute >= 9 and respiratory_rate_per_minute < 12:
    points += 1
  elif respiratory_rate_per_minute >= 12 and respiratory_rate_per_minute < 21:
    points += 0
  elif respiratory_rate_per_minute >= 21 and respiratory_rate_per_minute <= 24:
    points += 2
  elif respiratory_rate_per_minute > 24:
    points += 3


  #Oxygen saturation as a percent between 0 and 1
  if oxygen_saturation > (95 / 100):
    points += 0
  elif oxygen_saturation >= (94 / 100) and oxygen_saturation <= (95 / 100):
    points += 1
  elif oxygen_saturation >= (92 / 100) and oxygen_saturation < (94 / 100):
    points += 2
  elif oxygen_saturation < (92 / 100):
    points += 3


  # Heart rate in beats per minute
  if heart_rate < 41:
    points += 3
  elif heart_rate >= 41 and heart_rate < 51:
    points += 1
  elif heart_rate >= 51 and heart_rate < 91:
    points += 0
  elif heart_rate >= 91 and heart_rate < 111:
    points += 1
  elif heart_rate >= 111 and heart_rate <= 130:
    points += 2
  elif heart_rate > 130:
    points += 3


  #Systolic BP in mmHg
  if systolic_BP  < 91:
    points += 3
  elif systolic_BP >= 91 and systolic_BP < 101:
    points += 2
  elif systolic_BP >= 101 and systolic_BP < 111:
    points += 1
  elif systolic_BP >= 111 and systolic_BP <= 219:
    points += 0
  elif systolic_BP > 219:
    points += 3


  #Temperature in C
  if temperature < 35.1:
    points += 3
  elif temperature >= 35.1 and temperature < 36.1:
    points += 1
  elif temperature >= 36.1 and temperature < 38.1:
    points += 0
  elif temperature >= 38.1 and temperature <= 39.0:
    points += 1
  elif temperature > 39.0:
    points += 2


  #Alertness as a string description
  if alert not in ['alert', 'confused or not alert']:
    return None
  elif alert == 'alert':
    points += 0
  elif alert == 'confused or not alert':
    points += 3

  #Inspired Oxygen as as string description
  if inspired_oxygen not in ['air', 'supplemental oxygen']:
    return None
  elif inspired_oxygen == 'air':
    points += 0
  elif inspired_oxygen == 'supplemental oxygen':
    points += 2

  #Performance Status as a string description
  if performance_status not in ['unrestricted normal activity', 'limited strenuous activity, can do light activity',
                                'limited activity, can self-care', 'limited self-care', 'bed/chair bound, no self-care']:
    return None
  elif performance_status == 'unrestricted normal activity':
    points += 0
  elif performance_status == 'limited strenuous activity, can do light activity':
    points += 1
  elif performance_status == 'limited activity, can self-care':
    points += 2
  elif performance_status == 'limited self-care':
    points += 3
  elif performance_status == 'bed/chair bound, no self-care':
    points += 4

  #returning the pobability based on the rule provided above
  if points >= 0 and points < 2:
    return (1/100)
  elif points >= 2 and points < 4:
    return (2/100)
  elif points == 4:
    return (3/100)
  elif points == 5:
    return (9/100)
  elif points == 6:
    return (15/100)
  elif points == 7:
    return (18/100)
  elif points == 8:
    return (22/100)
  elif points == 9:
    return (26/100)
  elif points == 10:
    return (29/100)
  elif points == 11:
    return (34/100)
  elif points == 12:
    return (38/100)
  elif points == 13:
    return (46/100)
  elif points == 14:
    return (47/100)
  elif points == 15:
    return (49/100)
  elif points == 16:
    return (55/100)
  elif points >= 17 and points < 26:
    return 0.59
  elif points >= 26:
    return 0.99

In [100]:
print(priest('male', 50, 24, 0.80, 80, 120, 37, 'alert', 'room air', 'good'))

None


In [101]:
print(priest('male', 50, 24, 0.80, 80, 120, 37, 'alert', 'air', 'unrestricted normal activity'))

0.22


In [102]:
print(priest('male', 50, 24, 0.80, 80, 120, 37, 'confused or not alert', 'air', 'bed/chair bound, no self-care'))

0.49


In [103]:
print(priest('male', 50, 24, 0.80, 80, 120, 39.0, 'confused or not alert', 'air', 'bed/chair bound, no self-care'))

0.55


In [104]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 50, 24, 0.80, 80, 120, 37, 'alert', 'room air', 'good')
Expecting nothing
ok
Trying:
    priest('male', 50, 24, 0.80, 80, 120, 37, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.22
ok
Trying:
    priest('female', 66, 26, 0.90, 140, 80, 34, 'Confused or not alert', 'Supplemental oxygen', 'Bed/chair bound, no self-care')
Expecting:
    0.99
ok
Trying:
    priest('female', 66, 26, 0.90, 140, 80, 34, 'Confused or not alert', 'Supplemental oxygen', 'Unrestricted normal activity')
Expecting:
    0.59
ok
Trying:
    priest('female', 66, 26, 0.90, 140, 80, 34, 'Confused or not alert', 'Supplemental oxygenss', 'Unrestricted normal activity')
Expecting nothing
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [105]:
import requests as req
#understing the json file the url returns.
url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1"
response_body = req.get(url)
if response_body.status_code == 200:
  data = response_body.json()
  print(data)
else:
  print("No json")

{'age': '40', 'sex': 'male', 'risk': '0.1', 'hospital': 'Southwest Hospital and Medical Center'}


In [106]:
def find_hospital(age, sex, risk):
  """(int, str, str)->str
  Find the appropriate hospital based on age, sex, and risk percentage.
  Example:
  >>> find_hospital(40, 'male', 0.1)
  'Southwest Hospital and Medical Center'
  >>> find_hospital(60, 'female', 0.2)
  'Select Specialty Hospital - Northeast Atlanta'
  >>> find_hospital(30, 'malesd', 0.05)
  >>> find_hospital(20, 'male', 0.2)
  'Southwest Hospital and Medical Center'
  """
  sex = sex.lower()
  if sex not in ['male', 'female']:
    return None
  else:
    #using the url replacing the ?age= with the parameter on the function.
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"
    response_body = req.get(url)
    if response_body.status_code == 200:
      data = response_body.json()
      #print(data)
      return data.get('hospital')

In [107]:
#testing with the one above
print(find_hospital(40, 'male', 0.1))

Southwest Hospital and Medical Center


In [108]:
print(find_hospital(60, 'female', 0.2))


Select Specialty Hospital - Northeast Atlanta


In [109]:
print(find_hospital(30, 'malesd', 0.05))

None


In [110]:
print(find_hospital(20, 'male', 0.2))

Southwest Hospital and Medical Center


In [111]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(60, 'female', 0.2)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(30, 'malesd', 0.05)
Expecting nothing
ok
Trying:
    find_hospital(20, 'male', 0.2)
Expecting:
    'Southwest Hospital and Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [112]:
import json
#Loading to display the json file generated
url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
response = req.get(url)
json_data = response.text
print(json_data)

{
  "COOK MEDICAL CENTER":{
    "City":"ADEL",
    "TYPE":"GENERAL ACUTE CARE",
    "ADDRESS":"706 NORTH PARRISH AVENUE",
    "WEBSITE":"http:\/\/www.memorialofadel.com\/body.cfm?id=364",
    "BEDS":60,
    "NAICS_DESC":"GENERAL MEDICAL AND SURGICAL HOSPITALS"
  },
  "PHOEBE PUTNEY MEMORIAL HOSPITAL":{
    "City":"ALBANY",
    "TYPE":"GENERAL ACUTE CARE",
    "ADDRESS":"417 THIRD AVENUE",
    "WEBSITE":"http:\/\/phoebeputney.com\/PhoebeContentPage.aspx?nd=1523",
    "BEDS":691,
    "NAICS_DESC":"GENERAL MEDICAL AND SURGICAL HOSPITALS"
  },
  "PHOEBE PUTNEY MEMORIAL HOSPITAL - NORTH CAMPUS":{
    "City":"ALBANY",
    "TYPE":"GENERAL ACUTE CARE",
    "ADDRESS":"2000 PALMYRA ROAD",
    "WEBSITE":"http:\/\/phoebeputney.com\/PhoebeContentPage.aspx?nd=1643",
    "BEDS":248,
    "NAICS_DESC":"GENERAL MEDICAL AND SURGICAL HOSPITALS"
  },
  "BACON COUNTY HOSPITAL":{
    "City":"ALMA",
    "TYPE":"CRITICAL ACCESS",
    "ADDRESS":"302 SOUTH WAYNE STREET",
    "WEBSITE":"http:\/\/www.baconcountyho

In [113]:
def get_address(hospital_name):
  """(str)->str
  Get the address of a hospital based on its name.
  example:
  >>> get_address('COOK MEDICAL CENTER')
  '706 NORTH PARRISH AVENUE'
  >>> get_address('DOOLY MEDICAL CENTER')
  '1300 UNION STREET'
  >>> get_address('Southwest Hospital and Medical Center')
  '501 FAIRBURN ROAD SW'
  >>> get_address('Southwest Hospital and Medical Centerss')
  >>> get_address('HIGGINS GENERAL HOSPITAL')
  '200 ALLEN MEMORIAL DRIVE'
  """
  hospital_name = hospital_name.lower()
  hospitals = json.loads(json_data)
  for hospital in hospitals.items():
    if hospital[0].lower() == hospital_name:
      return hospital[1].get('ADDRESS')


In [114]:
print(get_address('Southwest Hospital and Medical Center'))
print(get_address('COOK MEDICAL CENTER'))
print(get_address('DOOLY MEDICAL CENTER'))
print(get_address('HIGGINS GENERAL HOSPITAL'))
print(get_address('HIGGINS GENERAL HOSPITALgftr'))

501 FAIRBURN ROAD SW
706 NORTH PARRISH AVENUE
1300 UNION STREET
200 ALLEN MEMORIAL DRIVE
None


In [115]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('COOK MEDICAL CENTER')
Expecting:
    '706 NORTH PARRISH AVENUE'
ok
Trying:
    get_address('DOOLY MEDICAL CENTER')
Expecting:
    '1300 UNION STREET'
ok
Trying:
    get_address('Southwest Hospital and Medical Center')
Expecting:
    '501 FAIRBURN ROAD SW'
ok
Trying:
    get_address('Southwest Hospital and Medical Centerss')
Expecting nothing
ok
Trying:
    get_address('HIGGINS GENERAL HOSPITAL')
Expecting:
    '200 ALLEN MEMORIAL DRIVE'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [116]:
def process_people(file_location):
  """(file)->dict
  Process the data from the file and determine hospital and address
 """
  response = req.get(file_location)
  data = response.text
  rows = data.strip().split('\n')
  #used to understand the structure of the file.
  #print(rows)
  header_skipped = False
  processed_data = {}
  for row in rows:
    #skipping the headers
    if not header_skipped:
      header_skipped = True
      continue
    patient, sex, age, breath, o2sat, heart, systolicbp, temp, alertness, inspired, status = row.split('|')
    # print(patient, sex, age, breath, o2sat, heart, systolicbp, temp, alertness, inspired, status)
    #converting the columns to correct data types
    sex = sex.lower()
    alertness = alertness.lower()
    inspired= inspired.lower()
    status = status.lower()
    age = int(age)
    breath = float(breath)
    o2sat = float(o2sat)
    heart = int(heart)
    systolicbp = float(systolicbp)
    temp = float(temp)

    risk = priest(sex, age, breath, o2sat, heart, systolicbp, temp, alertness, inspired, status)
    #print(f"Patient: {patient}, Risk: {risk}")
    if risk is not None:
      hospital = find_hospital(age, sex, risk)
      address = get_address(hospital)
      if hospital is not None:
        processed_data[patient] = [
            sex,
            age,
            breath,
            o2sat,
            heart,
            systolicbp,
            temp,
            alertness,
            inspired,
            status,
            risk,
            hospital,
            address]
  json_data = json.dumps(processed_data)
  return json_data


In [117]:
file_location = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
result = process_people(file_location)
print(result)

{"E9559": ["female", 40, 24.0, 0.96, 105, 115.0, 34.9, "alert", "air", "unrestricted normal activity", 0.15, "Select Specialty Hospital - Northeast Atlanta", "1821 CLIFTON ROAD NE"], "E9385": ["female", 51, 19.0, 0.99, 90, 91.0, 38.1, "alert", "supplemental oxygen", "unrestricted normal activity", 0.18, "Select Specialty Hospital - Northeast Atlanta", "1821 CLIFTON ROAD NE"], "E3067": ["female", 40, 29.0, 0.96, 105, 95.0, 38.1, "alert", "air", "unrestricted normal activity", 0.18, "Select Specialty Hospital - Northeast Atlanta", "1821 CLIFTON ROAD NE"], "E9422": ["female", 66, 19.0, 0.96, 135, 115.0, 38.1, "alert", "air", "bed/chair bound, no self-care", 0.34, "Wesley Woods Geriatric Hospital", "1821 CLIFTON ROAD, NE"], "E8661": ["male", 18, 22.0, 0.95, 105, 115.0, 36.3, "confused or not alert", "air", "unrestricted normal activity", 0.22, "Southwest Hospital and Medical Center", "501 FAIRBURN ROAD SW"], "E6235": ["male", 16, 23.0, 0.96, 135, 115.0, 39.2, "alert", "supplemental oxygen"

In [118]:
import doctest
doctest.run_docstring_examples(process_people, globals(),verbose=True)

Finding tests in NoName


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [119]:
url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
file_locations = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
response = req.get(url)
data = response.text
data_json = json.loads(data)
result = process_people(file_locations)
info = json.loads(result)

print(info)
print(data_json)

print('\n')

print(f"Keys in 'info': {list(info.keys())}")
print(f"Keys in 'data_json': {list(data_json.keys())}")
similar_keys = set(info.keys()) & set(data_json.keys())
print(f"Number of similar keys: {len(similar_keys)}")
print(f"Similar keys: {list(similar_keys)}")
not_similar_keys = set(info.keys()) ^ set(data_json.keys())
print(f"Number of not similar keys: {len(not_similar_keys)}")
print(f"Keys that are not similar: {list(not_similar_keys)}")

if info.keys() == data_json.keys():
  print("OK")
else:
  print("There is an error")

print('\n')

#modified the strings to lower cases
for key, value in data_json.items():
    value[0] = value[0].lower()
    value[7] = value[7].lower()
    value[8] = value[8].lower()
    value[9] = value[9].lower()

print(info)
print(data_json)

print('\n')

if info == data_json:
  print('The code is okay')
else:
  print('There is an error with the code')


{'E9559': ['female', 40, 24.0, 0.96, 105, 115.0, 34.9, 'alert', 'air', 'unrestricted normal activity', 0.15, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E9385': ['female', 51, 19.0, 0.99, 90, 91.0, 38.1, 'alert', 'supplemental oxygen', 'unrestricted normal activity', 0.18, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E3067': ['female', 40, 29.0, 0.96, 105, 95.0, 38.1, 'alert', 'air', 'unrestricted normal activity', 0.18, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E9422': ['female', 66, 19.0, 0.96, 135, 115.0, 38.1, 'alert', 'air', 'bed/chair bound, no self-care', 0.34, 'Wesley Woods Geriatric Hospital', '1821 CLIFTON ROAD, NE'], 'E8661': ['male', 18, 22.0, 0.95, 105, 115.0, 36.3, 'confused or not alert', 'air', 'unrestricted normal activity', 0.22, 'Southwest Hospital and Medical Center', '501 FAIRBURN ROAD SW'], 'E6235': ['male', 16, 23.0, 0.96, 135, 115.0, 39.2, 'alert', 'supplemental oxygen'

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---